## Loading the dataset and neural network classes from the respective notebooks using the _ipynb_ package

In [ ]:
from ipynb.fs.full.Dataset import *
from ipynb.fs.full.Network import *

## Defining the dataset and forming the training and validation DataLoaders

In [ ]:
#defining size of the dataset
n = 100000

case = 'original'
dens_cut_str = None
A_true = None
mult_factor = 1.0

#forming the dataset from the data generating class
data_set_train = data_gen(n, 'train', case, dens_cut_str, mult_factor, A_true)
train_dl = DataLoader(dataset=data_set_train, batch_size=128, shuffle=True)
print('Size of train dataset =', data_set_train.__len__())


data_set_valid = data_gen(n, 'valid', case, dens_cut_str, mult_factor, A_true)
valid_dl = DataLoader(dataset=data_set_valid, batch_size=128, shuffle=True)
print('Size of validation dataset =', data_set_valid.__len__())

Size of train dataset = 70000
Size of validation dataset = 15000


## Checking whether this code has GPU access and CUDA support, and if yes, how many GPUs can be used

In [ ]:
# use GPUs if available
import torch.backends.cudnn as cudnn

if torch.cuda.is_available():
    print("Using CUDA")
    device = torch.device('cuda')
else:
    print('CUDA Not Available')
    device1 = torch.device('cpu')
    
    
cudnn.benchmark = True      #May train faster but cost more memory

if (device == torch.device('cuda')):
    if torch.cuda.device_count() > 1:
        print("%d GPUs Available"%(torch.cuda.device_count()))
    print('GPU model:',torch.cuda.get_device_name())

else:
    print('Using CPU (Cuda unavailable)\n')


Using CUDA
3 GPUs Available
GPU model: Tesla V100-SXM2-32GB


## Defining the location to save the model weights and the _Optuna_ databases, and creating the directory if not already present

In [ ]:
if case == 'original':
    dir_wt = '/mnt/ceph/users/alahiry/power_spectra/{}'.format(case)
else:
    dir_wt = '/mnt/ceph/users/alahiry/power_spectra/{}/{}'.format(case, dens_cut_str)
if not os.path.exists(dir_wt):
    os.makedirs(dir_wt)  # create folder if it does not exist
    
print('Folder created: ' + dir_wt)

Folder created: /mnt/ceph/users/alahiry/power_spectra/original


## Main training and hyperparameter optimisation:

### (50 _Optuna_ trials and 200 epochs of training in each trial)

In [ ]:
def fit(params, epochs, model, train_dl, valid_dl, trial):

    min_valid_loss = 10**34

    lr = params['lr']
    wd = params['wd']
    
    loss_fn = F.mse_loss

    optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = wd)
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr = lr/100, max_lr = lr*100, cycle_momentum=False)

    for epoch in range(epochs):  # loop over the dataset multiple times
    
        #training model
        train_loss1 = torch.zeros(1).to(device)
        train_loss, counts = 0.0, 0
        model.train()
        for Pk, A_true in train_dl:
            bs = Pk.shape[0]
            Pk = Pk.to(device)
            A_true = A_true.to(device)
            A_nn = model(Pk)
            loss = loss_fn(A_nn, A_true)
            train_loss1 += loss*bs
            counts += bs
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
        train_loss = train_loss1/counts
        train_loss = torch.mean(train_loss).item()

    #evaluating with validation set
    valid_loss1 = torch.zeros(1).to(device)
    valid_loss, counts = 0.0, 0
    model.eval()
    for images, labels in valid_dl:
        with torch.no_grad():
            bs = Pk.shape[0]
            Pk = Pk.to(device)
            A_true = A_true.to(device)
            A_nn = model(Pk)
            loss = loss_fn(A_nn, A_true)
            valid_loss1 += loss*bs
            counts += bs
    valid_loss = valid_loss1/counts
    valid_loss = torch.mean(valid_loss).item()

    
    #saving the model if the loss is better(lower) than the previous epoch
    if valid_loss < min_valid_loss:
        fweights = dir_wt + '/weights_{}.pt'.format(trial.number)  
        torch.save(model.state_dict(), fweights)
        min_valid_loss = valid_loss
   
    floss = dir_wt + '/losses_{}.txt'.format(trial.number)  
    #saving epoch, training and validation losses to a text file
    f = open(floss, 'a')
    f.write('%d %.5e %.5e\n'%(epoch, train_loss, valid_loss))
    f.close()

    return min_valid_loss


def objective(trial):

    params = {
            'lr': trial.suggest_float("lr", 1e-6, 5e-3, log=True),
            'wd': trial.suggest_float('wd', 1e-7, 1e-1, log=True),
            'hidden': trial.suggest_int("hidden", 4, 256),
            'alpha': trial.suggest_float("alpha", 0.1, 0.9)
           }

    model = Model_Pk(params)

    # use GPUs if available
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    cudnn.benchmark = True      #May train faster but cost more memory


    if (device == torch.device('cuda')):
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
    model.to(device)
    
    epochs = 200
    best_loss = fit(params,epochs, model, train_dl, valid_dl, trial)

    return best_loss

if case == 'original':
    study_name = 'AstroNone_Pk_{}'.format(case)
    storage    = 'sqlite:///{}/AstroNone_Pk_{}.db'.format(dir_wt, case)
else:
    study_name = 'AstroNone_Pk_{}_{}'.format(case, dens_cut_str)
    storage    = 'sqlite:///{}/AstroNone_Pk_{}_{}.db'.format(dir_wt, case, dens_cut_str)
    
start = time.time()

study = optuna.create_study(direction = "minimize", sampler=optuna.samplers.TPESampler(), study_name = study_name, storage = storage, load_if_exists = True)
study.optimize(objective, n_trials = 50)

stop = time.time()

print('\nTime taken : {:.4f} hrs:'.format((stop-start)/3600.0))

## Checking the best trial parameters

In [ ]:
#Location og the optuna database
study_name = 'AstroNone_Pk_{}'.format('original')
storage    = 'sqlite:///{}/AstroNone_Pk_{}.db'.format(dir_wt, 'original')

#loading the study
study = optuna.load_study(study_name = study_name, storage = storage)

#checking the best trial parameters
best_trial = study.best_trial
best_trial_num = best_trial.number
print('\nBest model parameters:\n')
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))
print('Loss = ', best_trial.value)
params_final = best_trial.params
print('\nBest trial number: ', best_trial_num)


Best model parameters:

alpha: 0.13675798557499638
hidden: 159
lr: 4.102915788654486e-06
wd: 1.608970137425372e-07
Loss =  0.00011599007848417386

Best trial number:  39
